In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, confusion_matrix, accuracy_score, precision_score, f1_score

DETECTION_WEIGHT = 1
CLASSIFICATION_WEIGHT = 1

def print_metrics(df, confidence_col, prediction_col, exp_name):
    target = df['Target']
    conf = df[confidence_col]
    pred = df[prediction_col]
    
    print(f"Experiment: {exp_name}\n")
    print(f"AUROC: {roc_auc_score(target, conf).round(4)}")
    print(f"AUPRC: {average_precision_score(target, conf).round(4)}")
    print(f"Accuracy: {accuracy_score(target, pred).round(4)}")
    print(f"Precision: {precision_score(target, pred).round(4)}")
    print(f"Recall: {recall_score(target, pred).round(4)}")
    print(f"F1: {f1_score(target, pred).round(4)}")
    
    print("\nConfusion matrix")
    print(confusion_matrix(target, pred))

df_YOLOX = pd.read_csv("YOLOX/YOLOX_outputs/yolox_m_LVI/vis_res/2024_01_09_16_15_12/results.csv")

df_YOLOX.loc[:, "target"] = df_YOLOX.fname.map(lambda x: 1 if x.split("/")[-1].split("_")[0] == "pos" else 0)
df_YOLOX = df_YOLOX.drop_duplicates().reset_index(drop=True)

df_YOLOX.loc[:, "pred"] = df_YOLOX.max_score.map(lambda x: 1 if x >= 0.8 else 0)
df_YOLOX.fname = df_YOLOX.fname.map(lambda x: x.split("/")[-1])

df_YOLOX = df_YOLOX.loc[:, ["fname", "target", "max_score", "pred"]]
df_YOLOX.columns = ["Fname", "Target", "YOLOX_confidence", "YOLOX_prediction"]
df_YOLOX.head()

,Fname,Target,YOLOX_confidence,YOLOX_prediction
0,neg_A00000_10765-15516.png,0,0.9181,1
1,neg_A00000_16807-1873.png,0,0.0928,0
2,neg_A00000_17086-1885.png,0,0.0069,0
3,neg_A00000_1884-26418.png,0,0.0020,0
4,neg_A00000_20891-8601.png,0,0.0019,0


In [2]:
df_convit = pd.read_csv("ConViT/output/convit_inference.csv").loc[:, ["fname", "pred"]]
df_convit.columns = ["Fname", "ConViT_confidence"]

df_convit.loc[:, "ConViT_prediction"] = df_convit.ConViT_confidence.map(lambda x: 1 if x > 0.5 else 0)

df = df_YOLOX.merge(df_convit, on="Fname", how="inner")
df.loc[:, ["Ensemble_confidence", "Ensemble_prediction"]] = 0

df.Ensemble_confidence = (DETECTION_WEIGHT * df.YOLOX_confidence + CLASSIFICATION_WEIGHT * df.ConViT_confidence) / 2
df.Ensemble_prediction = df.Ensemble_confidence.map(lambda x: 1 if x >= 0.5 else 0)

df.head()

,Fname,Target,YOLOX_confidence,YOLOX_prediction,ConViT_confidence,ConViT_prediction,Ensemble_confidence,Ensemble_prediction
0,neg_A00000_10765-15516.png,0,0.9181,1,0.154440,0,0.536270,1
1,neg_A00000_16807-1873.png,0,0.0928,0,0.103115,0,0.097957,0
2,neg_A00000_17086-1885.png,0,0.0069,0,0.024377,0,0.015638,0
3,neg_A00000_1884-26418.png,0,0.0020,0,0.002812,0,0.002406,0
4,neg_A00000_20891-8601.png,0,0.0019,0,0.029424,0,0.015662,0


In [3]:
print_metrics(df, "YOLOX_confidence", "YOLOX_prediction", "Detection-YOLOX")

Experiment: Detection-YOLOX

AUROC: 0.8915
AUPRC: 0.835
Accuracy: 0.8859
Precision: 0.8194
Recall: 0.8113
F1: 0.8153

Confusion matrix
[[617  54]
 [ 57 245]]


In [4]:
print_metrics(df, "ConViT_confidence", "ConViT_prediction", "Classification-ConViT")

Experiment: Classification-ConViT

AUROC: 0.9184
AUPRC: 0.8726
Accuracy: 0.8674
Precision: 0.7781
Recall: 0.8013
F1: 0.7896

Confusion matrix
[[602  69]
 [ 60 242]]


In [5]:
print_metrics(df, "Ensemble_confidence", "Ensemble_prediction", "Ensemble")

Experiment: Ensemble

AUROC: 0.9438
AUPRC: 0.9167
Accuracy: 0.8983
Precision: 0.8372
Recall: 0.8344
F1: 0.8358

Confusion matrix
[[622  49]
 [ 50 252]]
